In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


os.environ["JAVA_HOME"] = "C:/Program Files/Eclipse Adoptium/jdk-17.0.17.10-hotspot"
# C:\Program Files\Eclipse Adoptium\jdk-17.0.17.10-hotspot\bin\java.exe
os.environ["SPARK_HOME"] = "C:/spark/spark-4.0.1-bin-hadoop3"
# C:\Users\jkuny\AppData\Local\Python\pythoncore-3.14-64\Scripts\spark-submit
os.environ["HADOOP_HOME"] = "C:/hadoop"
os.environ["PATH"] += ";" + os.path.join(os.environ["HADOOP_HOME"], "bin")

# spark-submit --version




In [2]:
# print(os.environ["HADOOP_HOME"])
# os.system("winutils.exe ls C:/tmp")  # ทดสอบว่า winutils รันได้

# os.system("C:/hadoop/bin/winutils.exe ls C:/tmp")



In [3]:
import findspark
findspark.init()


In [4]:
# from pyspark.sql import SparkSession
# C:\Users\jkuny>where spark-submit
# C:\Users\jkuny\AppData\Local\Python\pythoncore-3.14-64\Scripts\spark-submit
# C:\Users\jkuny\AppData\Local\Python\pythoncore-3.14-64\Scripts\spark-submit.cmd

spark = (
    SparkSession.builder
    .appName("weather_traffy")
    .master("local[*]")   # ใช้ทุก core ของเครื่อง
    .config("spark.driver.bindAddress", "127.0.0.1") \
    # .config("spark.sql.warehouse.dir", r"C:\Users\jkuny") \
    .config("spark.sql.warehouse.dir", "D:/CEDT/02_01/data_Sci/dsde-final/data") \
    .getOrCreate()
)


In [5]:
# path = # ตัวอย่าง absolute path
path = '../../data/raw/open-meteo-13.74N100.50E9m.csv'


df_weather = spark.read.csv(path, header=True, inferSchema=True)
df_weather.show()

+-------------------+-------------------+-----------------+------------------------+---------+-----------------------------+---------------+----------------------+----------------------+---------------------+
|               time|temperature_2m (°C)|dew_point_2m (°C)|relative_humidity_2m (%)|rain (mm)|vapour_pressure_deficit (kPa)|cloud_cover (%)|wind_direction_10m (°)|surface_pressure (hPa)|wind_speed_10m (km/h)|
+-------------------+-------------------+-----------------+------------------------+---------+-----------------------------+---------------+----------------------+----------------------+---------------------+
|2021-08-01 00:00:00|               27.2|             24.4|                      84|      0.0|                         0.56|             65|                   215|                1006.1|                  5.7|
|2021-08-01 01:00:00|               28.8|             24.5|                      78|      0.0|                         0.89|             90|                   207| 

In [6]:

df_traffyM = spark.read.csv('../../data/processed/traffy_merged.csv', header=True, inferSchema=True)
# D:\CEDT\02_01\data_Sci\dsde-final\data\processed\traffy_merged.csv
df_traffyM.show()

+--------------------+--------------------+------------------+-----------+-------------+--------------------+--------------------+----------+-----+-----+----+------------------+--------------------+
|                type|             comment|            coords|subdistrict|     district|            province|           timestamp|      date| pm25| pm10|  o3|               no2|           type_list|
+--------------------+--------------------+------------------+-----------+-------------+--------------------+--------------------+----------+-----+-----+----+------------------+--------------------+
| {น้ำท่วม,ร้องเรียน}|น้ำท่วมเวลาฝนตกแล...|100.66709,13.67891|    หนองบอน|       ประเวศ|       กรุงเทพมหานคร|2021-09-19 14:56:...|2021-09-19|   52|   27|   9|                 4|['น้ำท่วม', 'ร้อง...|
|             {สะพาน}|สะพานลอยปรับปรุงไ...|              NULL|       NULL|         NULL|                NULL|                NULL|      NULL| NULL| NULL|NULL|              NULL|                NULL|
|    

In [7]:
from pyspark.sql.functions import expr, date_trunc

# df_traffyM: สร้าง column timestamp_safe แบบ try_cast
df_traffyM = df_traffyM.withColumn(
    "timestamp_safe",
    expr("try_cast(trim(timestamp) AS timestamp)")
)

# row ที่ cast ไม่ได้
df_invalid = df_traffyM.filter(col("timestamp_safe").isNull())
df_invalid.show(truncate=False)

# ตัด minutes/seconds
df_traffyM = df_traffyM.withColumn(
    "date_time_truncated",
    date_trunc("hour", col("timestamp_safe"))
)

# df_weather
df_weather = df_weather.withColumn(
    "time_safe",
    expr("try_cast(trim(time) AS timestamp)")
)


+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------+------+-----------+--------+--------+---------+----+-----+-----+----+----+---------+--------------+
|type                                                                                                                                                                                                                  |comment                                                                                                   |coords|subdistrict|district|province|timestamp|date| pm25| pm10| o3 | no2|type_list|timestamp_safe|
+---------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
df_traffyM_clean = df_traffyM.filter(col("date_time_truncated").isNotNull())
df_weather_clean = df_weather.filter(col("time_safe").isNotNull())


Merge and export

In [9]:
merged_df = df_traffyM_clean.join(
    df_weather_clean,
    df_traffyM_clean.date_time_truncated == df_weather_clean.time_safe,
    how="outer"
)

In [17]:
merged_df.show()

+----+-------+------+-----------+--------+--------+---------+----+-----+-----+----+----+---------+--------------+-------------------+-------------------+-------------------+-----------------+------------------------+---------+-----------------------------+---------------+----------------------+----------------------+---------------------+-------------------+
|type|comment|coords|subdistrict|district|province|timestamp|date| pm25| pm10|  o3| no2|type_list|timestamp_safe|date_time_truncated|               time|temperature_2m (°C)|dew_point_2m (°C)|relative_humidity_2m (%)|rain (mm)|vapour_pressure_deficit (kPa)|cloud_cover (%)|wind_direction_10m (°)|surface_pressure (hPa)|wind_speed_10m (km/h)|          time_safe|
+----+-------+------+-----------+--------+--------+---------+----+-----+-----+----+----+---------+--------------+-------------------+-------------------+-------------------+-----------------+------------------------+---------+-----------------------------+---------------+------

In [ ]:
# from pyspark.sql.functions import col, expr

# # แปลง column ด้วย try_cast
# merged_df = merged_df.withColumn(
#     "date_time_truncated",
#     expr("try_cast(date_time_truncated AS timestamp)")
# ).withColumn(
#     "time_safe",
#     expr("try_cast(time AS timestamp)")
# )

# # ลบ row ถ้า column ไหนเป็น null ก็ลบ
# merged_df = merged_df.filter(
#     col("date_time_truncated").isNotNull() & col("time_safe").isNotNull()
# )

# merged_df = merged_df.filter(
#     col("date_time_truncated")=="NULL" & col("time_safe")=="NULL"
# )


from pyspark.sql.functions import when, col, expr

merged_df = merged_df.withColumn(
    "date_time_truncated",
    when(col("date_time_truncated") == "NULL", None)
    .otherwise(col("date_time_truncated"))
).withColumn(
    "time_safe",
    when(col("time") == "NULL", None)
    .otherwise(col("time"))
)

merged_df = merged_df.withColumn(
    "date_time_truncated",
    expr("try_cast(date_time_truncated AS timestamp)")
).withColumn(
    "time_safe",
    expr("try_cast(time_safe AS timestamp)")
)



# ลบ row ที่เป็น null
merged_df = merged_df.filter(
    col("date_time_truncated").isNotNull() & col("time_safe").isNotNull()
)





# นับ row หลัง clean
print("Number of rows after clean:", merged_df.count())


# ถ้าต้องการบันทึกเป็น CSV
# merged_df_clean.write.csv("merged_df_clean.csv", header=True, mode="overwrite")


{"ts": "2025-11-27 20:23:30.419", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[CAST_INVALID_INPUT] The value 'NULL' of the type \"STRING\" cannot be cast to \"TIMESTAMP\" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018", "context": {"file": "line 31 in cell [23]", "line": "", "fragment": "__eq__", "errorClass": "CAST_INVALID_INPUT"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o403.count.\n: org.apache.spark.SparkDateTimeException: [CAST_INVALID_INPUT] The value 'NULL' of the type \"STRING\" cannot be cast to \"TIMESTAMP\" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018\n== DataFrame ==\n\"__eq__\" was called from\nline 31 in cell [23]\n\r\n\tat org.apache.spark.sql.errors.Execution

DateTimeException: [CAST_INVALID_INPUT] The value 'NULL' of the type "STRING" cannot be cast to "TIMESTAMP" because it is malformed. Correct the value as per the syntax, or change its target type. Use `try_cast` to tolerate malformed input and return NULL instead. SQLSTATE: 22018
== DataFrame ==
"__eq__" was called from
line 31 in cell [23]


In [18]:
pdf = merged_df.toPandas()
pdf.to_csv(r"D:\CEDT\02_01\data_Sci\dsde-final\data\merge\outer.csv", index=False)
pdf.to_excel(r"D:\CEDT\02_01\data_Sci\dsde-final\data\merge\outer.xlsx", index=False)


Py4JJavaError: An error occurred while calling o76.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 46.0 failed 1 times, most recent failure: Lost task 5.0 in stage 46.0 (TID 266) (puppy executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$3(DAGScheduler.scala:2935)
	at scala.Option.getOrElse(Option.scala:201)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2935)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2927)
	at scala.collection.immutable.List.foreach(List.scala:334)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2927)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1295)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1$adapted(DAGScheduler.scala

In [10]:

# test_path = "D:/CEDT/02_01/data_Sci/dsde-final/data/merge/test_write.txt"

# try:
#     with open(test_path, "w") as f:
#         f.write("test")
#     print("เขียนไฟล์ได้ปกติ")
#     os.remove(test_path)  # ลบไฟล์หลังทดสอบ
# except Exception as e:
#     print("ไม่สามารถเขียนไฟล์ได้:", e)


In [16]:
# output_path = "file:///D:/CEDT/02_01/data_Sci/dsde-final/data/merge/merged_df_outer"
output_path = r"D:\CEDT\02_01\data_Sci\dsde-final\data\merge\merged_df_outer"

merged_df.write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv(output_path)


# merged_df.write.mode("overwrite").parquet(output_path)



Py4JJavaError: An error occurred while calling o102.csv.
: java.util.concurrent.ExecutionException: Boxed Exception
	at scala.concurrent.impl.Promise$.scala$concurrent$impl$Promise$$resolve(Promise.scala:99)
	at scala.concurrent.impl.Promise$DefaultPromise.tryComplete(Promise.scala:278)
	at scala.concurrent.Promise.complete(Promise.scala:57)
	at scala.concurrent.Promise.complete$(Promise.scala:56)
	at scala.concurrent.impl.Promise$DefaultPromise.complete(Promise.scala:104)
	at scala.concurrent.Promise.failure(Promise.scala:109)
	at scala.concurrent.Promise.failure$(Promise.scala:109)
	at scala.concurrent.impl.Promise$DefaultPromise.failure(Promise.scala:104)
	at org.apache.spark.sql.execution.adaptive.ResultQueryStageExec.$anonfun$doMaterialize$2(QueryStageExec.scala:333)
	at java.base/java.util.concurrent.CompletableFuture.uniWhenComplete(CompletableFuture.java:863)
	at java.base/java.util.concurrent.CompletableFuture$UniWhenComplete.tryFire(CompletableFuture.java:841)
	at

In [ ]:
merged_df = df_traffyM_clean.join(
    df_weather_clean,
    df_traffyM_clean.date_time_truncated == df_weather_clean.time_safe,
    how="left"
)



# merged_df.csv("merged_df_left.csv", header=True, mode="overwrite")

# merged_df.show()

In [ ]:
output_path = "file:///D:/CEDT/02_01/data_Sci/dsde-final/data/merge/merged_df_left"

merged_df.write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv(output_path)



Py4JJavaError: An error occurred while calling o223.csv.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 31.0 failed 1 times, most recent failure: Lost task 0.0 in stage 31.0 (TID 91) (puppy executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$3(DAGScheduler.scala:2935)
	at scala.Option.getOrElse(Option.scala:201)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2935)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2927)
	at scala.collection.immutable.List.foreach(List.scala:334)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2927)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1295)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1$adapted(DAGScheduler.scala:1295)
	at 

In [ ]:
merged_df = df_traffyM_clean.join(
    df_weather_clean,
    df_traffyM_clean.date_time_truncated == df_weather_clean.time_safe,
    how="outer"
)


# Export merged_df_clean เป็น CSV
merged_df.write.csv(
    path="data/merge/merged_df_full.csv",  
    header=True,             
    mode="overwrite"            
)


Py4JJavaError: An error occurred while calling o392.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:314)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:1116)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:798)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:838)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:810)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:837)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(Ra

In [ ]:
merged_df.count()


659450

# Clean data

In [ ]:
from pyspark.sql.functions import expr, col,date_format

# แปลง column ทั้งคู่ด้วย try_cast ก่อนแสดงผล
merged_df = merged_df.withColumn(
    "date_time_truncated_safe",
    expr("try_cast(date_time_truncated AS timestamp)")
).withColumn(
    "time_safe",
    expr("try_cast(time AS timestamp)")
)

# ลบ row ที่ยัง cast ไม่ได้
merged_df = merged_df.filter(
    col("date_time_truncated_safe").isNotNull() &
    col("time_safe").isNotNull()
)



# นับ row หลัง clean
print("Number of rows after clean:", merged_df.count())


# ถ้าต้องการบันทึกเป็น CSV
# merged_df_clean.write.csv("merged_df_clean.csv", header=True, mode="overwrite")


Py4JJavaError: An error occurred while calling o352.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 70.0 failed 1 times, most recent failure: Lost task 0.0 in stage 70.0 (TID 200) (puppy executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$3(DAGScheduler.scala:2935)
	at scala.Option.getOrElse(Option.scala:201)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2935)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2927)
	at scala.collection.immutable.List.foreach(List.scala:334)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2927)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1295)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1$adapted(DAGScheduler.scala:129